In [1]:
!pip install numpy scipy networkx

Generating synthetic data

In [2]:
import numpy as np
from scipy.stats import multivariate_normal
from scipy.linalg import eigh


In [4]:
def generate_time_varying_precision(p=10, N=50, eps=1e-6, soft_lambda=0.14):
    """
    Generate a list of time-varying precision matrices Omega(t),
    following the four steps (i)-(iv) described:

      (i)  B_i are lower-triangular with entries ~ N(0,1/2).
      (ii) G(t) = ( sum_i B_i * phi_i(t) ) / 2.
      (iii) Omega^o(t) = G(t)G(t)^T, then soft-threshold off-diagonals.
      (iv) Add log10(p)/4 to the diagonal for positive definiteness.
    """

    # 1. Generate lower-triangular matrices B1,B2,B3,B4
    #    with elements from Normal(0, 1/2).
    B_list = [np.tril(np.random.normal(0, np.sqrt(0.5), (p, p))) 
              for _ in range(4)]

    # 2. Create time grid from 0 to 1
    t_values = np.linspace(0, 1, N)
    Omega_list = []

    for t in t_values:
        # Define phi_1(t), phi_2(t), phi_3(t), phi_4(t)
        phi = [
            np.sin(np.pi * t / 2),
            np.cos(np.pi * t / 2),
            np.sin(np.pi * t / 4),
            np.cos(np.pi * t / 4)
        ]

        # 3. Compute G(t) = (B1*phi1 + B2*phi2 + B3*phi3 + B4*phi4) / 2
        G = sum(B * ph for B, ph in zip(B_list, phi)) / 2

        # Omega^o(t) = G(t)*G(t)^T
        Omega_o = G @ G.T  # automatically symmetric

        # 4. Soft-threshold off-diagonal elements
        #    off_diag = sign(x) * max(|x|-lambda, 0)
        off_diag = Omega_o - np.diag(np.diag(Omega_o))  # zero out diag
        sign_off = np.sign(off_diag)
        mag_off = np.abs(off_diag)
        off_diag_thresh = sign_off * np.maximum(mag_off - soft_lambda, 0.0)

        # Put thresholded off-diagonals back, keep original diagonal
        Omega = np.diag(np.diag(Omega_o)) + off_diag_thresh

        # 5. Add log10(p)/4 to the diagonal to ensure positivity
        diag_adj = np.log10(p)/4 + eps
        Omega += np.eye(p) * diag_adj

        # 6. Double-check positive definiteness (just in case)
        eigvals, _ = eigh(Omega)
        if np.any(eigvals <= 0):
            # If not PD, shift up by |min_eig| + eps
            Omega += np.eye(p) * (abs(np.min(eigvals)) + eps)

        Omega_list.append(Omega)

    return Omega_list

def generate_synthetic_data(p=10, N=50):
    """
    Generate a time-series dataset X(t) by sampling from
    N(0, Sigma(t)), where Sigma(t) = Omega(t)^{-1}.
    The code returns:
      dataset: shape (N, p)
      Omega_list: list of precision matrices for each time grid
    """
    Omega_list = generate_time_varying_precision(p, N)

    dataset = np.zeros((N, p))
    for i, Omega in enumerate(Omega_list):
        # Invert each Omega(t) to get Sigma(t)
        try:
            Sigma = np.linalg.inv(Omega)
        except np.linalg.LinAlgError:
            Sigma = np.linalg.pinv(Omega)

        # Draw a single sample from N(0, Sigma)
        dataset[i] = multivariate_normal.rvs(
            mean=np.zeros(p),
            cov=Sigma,
            size=1
        )

    # Center the entire dataset
    dataset -= dataset.mean(axis=0)
    return dataset, Omega_list


Blockwise ADMM